# Exp.1 Train MLP as a baseline model

- Dataset: ASCADv1 fixed-key  
    Boolean masking (2-share), software-implementation
- PoI: 700 pts, specified by ASCAD team (ASCAD.h5)
- #traces: 50,000 (profiling), 20,000 (attack)
- Preprocess: Z-score normalization
- #attack traces to calculate GE: 2,000
- Labeling: LSB of unmasked SBox output (two class classification)
- Loss function: Softmax cross entropy (torch.nn.BCEWithLogitsLoss)
- Model architecture: [700(Input), 500(Tanh)*6, 2(Softmax)]  
    Most accurate MLP on ASCAD paper (NOT $MLP_{best}$)  
    #output layer nodes are changed to 2.

In [ ]:
import torch
from pathlib import Path
import os
import hydra
import numpy as np
import matplotlib.pyplot as plt
import pickle

os.chdir('/workspace')
import src

In [ ]:
with hydra.initialize(config_path="../conf", version_base='1.1'):
    cfg = hydra.compose(
        config_name='config',
        overrides=[
            "model=MLP_ASCAD",
            "model.train_params.steps=30000",
            "dataset@train=ASCADf_profiling",
            "dataset@test=ASCADf_attack",
            "label_transforms=bit",
            "label_transforms.transforms.3.pos=0",
            "trace_transforms=void",
            "save_path=/workspace/notebook/results/exp1/",
            "n_attack_traces=2000",
            ]
        )

In [ ]:
device = hydra.utils.instantiate(cfg.device)
cpu = torch.device('cpu')

In [ ]:
profiling_dataset = hydra.utils.instantiate(cfg.train.dataset)
test_dataset = hydra.utils.instantiate(cfg.test.dataset)

train_dataloader = torch.utils.data.DataLoader(
    profiling_dataset, batch_size=cfg.train.batch_size, shuffle=True
)
test_dataloader = torch.utils.data.DataLoader(
    test_dataset, batch_size=cfg.test.batch_size, shuffle=False
)

In [ ]:
model = hydra.utils.instantiate(cfg.model.model)
model = model.to(device)

In [ ]:
train_kwargs = hydra.utils.instantiate(cfg.model.train_params)
train_kwargs.opt = train_kwargs.opt(model.parameters())
_ = src.trainDNN.train(
    model,
    train_dataloader,
    test_dataloader,
    **train_kwargs
    )
model = model.to(cpu)

In [ ]:
print(
    f'[INFO] Save trained model to {cfg.save_path}/{cfg.model_name}.pt')
Path(cfg.save_path).mkdir(exist_ok=True, parents=True)
torch.save(model.state_dict(), Path(cfg.save_path, cfg.model_name+'.pt'))

In [ ]:
preds, labels, th = src.utils.make_prediction(
    model, test_dataloader, device,
    cfg.label_transforms.one_hot)
preds_class = np.argmax(preds, axis=1)
accuracy = np.mean(labels == preds_class)
print(accuracy)

In [ ]:
preds, labels, th = src.utils.make_prediction(
    model, test_dataloader, device,
    cfg.label_transforms.one_hot)
correct_key = test_dataset.key[0][cfg.target_byte]
key_hyposesis = range(256)
if not Path(cfg.save_path, 'label_hyposesis.npy').exists():
    label_hyposesis = src.utils.make_label_hyposesis(
        test_dataset, key_hyposesis, one_hot=cfg.label_transforms.one_hot)
    np.save(Path(cfg.save_path, 'label_hyposesis.npy'), label_hyposesis)
label_hyposesis = np.load(Path(cfg.save_path, 'label_hyposesis.npy'))
ge = src.sca_utils.calc_guessing_entropy(
    preds, label_hyposesis, correct_key,
    cfg.n_attack_traces, n_trial=cfg.n_trials)

fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(ge)
ax.set_title(f'GE@{cfg.n_attack_traces}:{ge[-1]}')
fig.savefig(Path(cfg.save_path, f'{cfg.model.name}.png'), dpi=300)